## DECORATOR (PEP318)
*python을 또 아름답게*

decorator는 보통 다음의 경우에 사용될 수 있습니다.
- 함수 호출 전, 호출 후 시점에 로직을 넣고 싶다면.
- 함수 arguments를 검사하고 싶을 때
- 함수 return값을 검사하고 싶을 때

어떻게 만드나요?
- 함수를 인자로 받아서 함수를 리턴하는 함수를 만들면 됩니다.

In [1]:
# 이것이 바로 함수를 인자로 받아서 함수를 리턴하는 함수
def decorator(func):
    return func

이렇게 사용합니다.

In [10]:
@decorator
def function(arg):
    return "Return {}".format(arg)

# decorator는 특별히 하는 일이 없으므로
function(1)

'Return 1'

In [3]:
def function(arg):
    return "Return {}".format(arg)

# 이렇게 표현하는 것과 같습니다.
function=decorator(function)

# 같은 결과값
function(1)

'Return 1'

그럼 함수 호출 전 로직을 넣어볼까요?

In [3]:
def decorator(func):
    print('before function call') # 함수 수행전 호출
    return func

In [4]:
@decorator
def function(arg):
    return 'Return {}'.format(arg)

before function call


<span style='color:red'>???</span>
무언가 틀렸습니다.
decorator는 <u>함수생성시점</u>에 호출되므로 위와 같이 정리하면 수행 전이긴 한데, 너무 일찍 수행되어 버립니다.

In [6]:
# 아까는 return전 호출이었을뿐,
# 수행(호출)전에 수행하려면 다음과 같이 nested function을 사용해야 합니다.
def decorator(func):
    def inner(*args, **kwargs):
        print('before function call')
        func(*args, **kwargs)
        print('after function call')
    return inner

@decorator
def function(arg):
    print('Return {}'.format(arg)) 

function(1)
# function = decorator(function)

before function call
Return 1
after function call


(잠깐!) decorator는 func 변수를 enclosing하는 closure입니다. (func가 자유변수) 뭐 의식하고 사용할 필요는 없지만 복습할 겸.

In [11]:
function.__code__.co_freevars # 위의 코드에서 func이 자유변수입니다.

('func',)

In [9]:
# dir(function)

@decorator
def function1(arg):
    print('{}'.format(arg))
@decorator
def function2(arg):
    print('{}'.format(arg))

# 함수는 내부에 이름, doc과 같은 meta값을 갖습니다.
(function1.__name__,
 function2.__name__)

('inner', 'inner')

In [10]:
import functools
def decorator(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print('before function call')
        func(*args, **kwargs)
        print('after function call')
    return inner

@decorator
def function1(arg):
    """xxx"""
    print('{}'.format(arg))
@decorator
def function2(arg):
    """yyy"""
    print('{}'.format(arg))

(function1.__name__, function2.__name__)

('function1', 'function2')

혹시 만들어야 할 일이 있을때 참고해서 쓰면 됩니다 :)
평소에는 만들어진 것을 잘 쓰는게 중요하죠!

## Parameterized Decorators
*좀 더 유명해지고 싶은 library들은 지금까지의 simple한 decorator로는 만족할 수 없었던 것 같습니다.*

In [17]:
def repeater(func):
    def inner(*args, **kwds):
        func(*args, **kwds)
        func(*args, **kwds)
    return inner

@repeater
def print_two():
    print(2)
print_two()

2
2


아래와 같은 식으로 개선해서 사용하고 싶은거죠.

In [12]:
import functools

def repeater(times):
    def repeater_times(func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            for i in range(times):
                func(*args, **kwargs)
        return inner
    return repeater_times

In [13]:
@repeater(3)
def print_two():
    print(2)
print_two()

@repeater(5)
def print_three():
    print(3)
print_three()

2
2
2
3
3
3
3
3


역시 필요할때 참고해서 쓰면 됩니다 :) template처럼 기억하세요.

## Stacked Decorators

In [20]:
def d1(func):
    return func
def d2(func):
    return func

# 아래와 같이 여러개를 겹쳐서 사용할 수 있습니다.
@d1
@d2
def hello():
    print('world')
hello()

def hello():
    print('world')

# 같은효과, 실행순서를 확인하세요.
hello = d1(d2(hello))
hello()

world
world


### CLASS DECORATOR (PEP3129)

function decorator와 거의 비슷합니다. cls를 취하고, cls를 return해주는 function을 만들면 됩니다.

In [45]:
def add_chirp(cls):
    def chirp(self):
        return 'CHIRP'
    cls.speak = chirp
    return cls

@add_chirp
class Bird:
    def speak(self):
        return '삐약'

bird = Bird()
bird.speak()

'CHIRP'

class에 부가기능을 붙이거나,
기존 라이브러리에 간단한 patch를 적용하여 쓰고 싶을때 사용할 수 있습니다.

standard library에서는 아래 6개 정도가 decorator로 많이 쓰입니다.

- property
- classmethod
- staticmethod
- functools.wraps (위에서 배운 것)
- functools.lru_cache
- functools.singledispatch

<u>다음의 일들을 위해서 decorator를 사용해볼 수 있습니다.</u>

- 고비용의 작업들을 caching하고 싶다면
- 함수가 동작실패했을 때 재시도를 위해서
- 함수가 내부적으로 표준출력으로 내보내는 것을 redirection하고 싶다면
- 함수가 동작한 시간을 logging해보려면
- 함수 동작시간을 통제하고 싶다면
- 접근제어를 위해서

## Exercise

다음 기능을 수행하는 decorator 두 개를 만들어봅시다.
 - input과 output을 logging하는 decorator
 - input이 문자가 아니거나 혹은 False일 경우 critical로 경고를 띄워주고 함수는 수행하지 않는 decorator

```
@check
@monitor
def hello(world):
    return len(world)

> hello("world")
[info] input string is "world"
[info] output value is 5
> hello("")
[critical] invalid input. done nothing.
> hello(3)
[critical] invalid input. done nothing.
```

In [13]:
import functools
import logging

def monitor(func):
    @functools.wraps(func)
    def inner2(*args, **kwargs):
        print(kwargs)
        logging.critical(f'input: {args}, {kwargs}')
        ret = func(*args, **kwargs)
        logging.critical(f'output: {ret}')
        return ret
    inner2.__name__ = func.__name__
    inner2.__doc__ = func.__doc__
    return inner2

@monitor
def hello(world, world2, extra='hello'):
    '''world의 길이를 return합니다.'''
    return len(world)

hello('world', 'world2', extra='hello3'), hello.__name__, hello.__doc__

CRITICAL:root:input: ('world', 'world2'), {'extra': 'hello3'}
CRITICAL:root:output: 5


{'extra': 'hello3'}


(5, 'hello', 'world의 길이를 return합니다.')